In [1]:
import glob 
from torchvision.transforms.v2 import ToTensor
import torchvision.transforms.v2 as transforms
from torchvision.io import decode_image
from torchvision.transforms.v2.functional import to_image
import torch 
from torch.utils.data import Dataset, DataLoader 
from tqdm import tqdm
import os
from torchvision import models
import torch.nn as nn
from PIL import Image
import pillow_heif
pillow_heif.register_heif_opener()

In [2]:
test_transform = transforms.Compose([  
    transforms.Resize((224, 224)), 
    transforms.ToImage(),
    transforms.ToDtype(torch.float32, scale=True),
    transforms.Normalize(   [0.485, 0.456, 0.406], 
                            [0.229, 0.224, 0.225])
])


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")

Using cuda device


# Evaluation phase

In [4]:
# model = models.resnet34(weights=None)
# num_ftrs = model.fc.in_features
# model.fc = nn.Linear(num_ftrs, 10)  # 10 classes for digits (0-9)

# weights_path = r'C:/Users/USER/Documents/MachineLearning/handwritten_digit/pretrained/non_aug_resnet34_digits_weights.pth'
# state_dict = torch.load(weights_path, weights_only=True)

# from collections import OrderedDict
# new_state_dict = OrderedDict()
# for k, v in state_dict.items():
#     name = k[7:] if k.startswith('module.') else k
#     new_state_dict[name] = v

# # Load the modified state dict
# model.load_state_dict(new_state_dict)
# model = model.to(device)

In [5]:
# model = models.resnet18(weights=None)
# num_ftrs = model.fc.in_features
# model.fc = nn.Linear(num_ftrs, 10)  # 10 classes for digits (0-9)

# weights_path = r'C:/Users/USER/Documents/MachineLearning/handwritten_digit/pretrained/non_aug_resnet18_digits_weights.pth'
# state_dict = torch.load(weights_path, weights_only=True)

# from collections import OrderedDict
# new_state_dict = OrderedDict()
# for k, v in state_dict.items():
#     name = k[7:] if k.startswith('module.') else k
#     new_state_dict[name] = v

# # Load the modified state dict
# model.load_state_dict(new_state_dict)
# model = model.to(device)

In [6]:
# model = models.efficientnet_b1(weights=None)
# num_ftrs = model.classifier[1].in_features
# model.classifier = nn.Sequential(
#     nn.Dropout(p=0.3, inplace=True),
#     nn.Linear(in_features=num_ftrs, out_features=10)  # 10 digits
# )
# weights_path = r'C:\Users\USER\Documents\MachineLearning\handwritten_digit\pretrained\non_aug_effb1_digits_weights.pth'

# state_dict = torch.load(weights_path, weights_only=True)

# # Load the modified state dict
# model.load_state_dict(state_dict)
# model = model.to(device)

In [13]:
model= models.vgg11(weights=None)
num_ftrs = model.classifier[6].in_features
model.classifier[6] = nn.Linear(num_ftrs, 10)
weights_path = r'C:\Users\USER\Documents\MachineLearning\handwritten_digit\pretrained\non_aug_vgg11_digits_weights.pth'

state_dict = torch.load(weights_path, weights_only=True)

from collections import OrderedDict
new_state_dict = OrderedDict()
for k, v in state_dict.items():
    name = k[7:] if k.startswith('module.') else k
    new_state_dict[name] = v

# Load the modified state dict
model.load_state_dict(new_state_dict)
model = model.to(device)

In [14]:
unlabeled_dir = r'C:\Users\USER\Documents\MachineLearning\handwritten_digit\data.2025'
unlabeled_images = []

for root, dirs, files in os.walk(unlabeled_dir):
    for file in files:
        if file.lower().endswith(('.jpg', '.jpeg', '.png', '.heic', '.jfif', '.md')):
            unlabeled_images.append(os.path.join(root, file))
        else:
            print(f"Skipped file: {file}")

print(f"Found {len(unlabeled_images)} images.")

Found 9998 images.


In [15]:
class CustomDataset(Dataset):
    def __init__(self, image_paths, transform=None):
        self.image_paths = image_paths
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        path = self.image_paths[idx]
        try:
            if path.lower().endswith('.md'):
                placeholder = torch.zeros((3, 224, 224), dtype=torch.float32)
                return placeholder, os.path.basename(path)
            
            file_size = os.path.getsize(path)
            if file_size < 10:  
                placeholder = torch.zeros((3, 224, 224), dtype=torch.float32)
                return placeholder, os.path.basename(path)
            
            else:
                image = Image.open(path).convert("RGB")
                image = to_image(image)
                if self.transform:
                    image = self.transform(image)
                return image, os.path.basename(path)
            
        except Exception as e:
            print(f"Error processing {path}: {str(e)}")
            placeholder = torch.zeros((3, 224, 224), dtype=torch.float32)
            return placeholder, os.path.basename(path)

In [16]:
eval_dataset = CustomDataset(unlabeled_images, transform=test_transform)
eval_loader = DataLoader(eval_dataset, batch_size=64, shuffle=False, num_workers=0)

In [17]:
model.eval()
predictions = []
filenames = []

with torch.no_grad():
    for images, image_paths in tqdm(eval_loader):
        images = images.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        
        predictions.extend(preds.cpu().numpy())
        filenames.extend(image_paths)

100%|██████████| 157/157 [05:24<00:00,  2.07s/it]


In [18]:
import pandas as pd

results = pd.DataFrame({
    'Filename': filenames,
    'Prediction': predictions
})

output_path = r'C:\\Users\\USER\\Documents\\MachineLearning\\handwritten_digit\\predictions\\predictionsvgg11_non_aug.csv'
results.to_csv(output_path, header=False, index=False)

print(f"Predictions saved to {output_path}")

Predictions saved to C:\\Users\\USER\\Documents\\MachineLearning\\handwritten_digit\\predictions\\predictionsvgg11_non_aug.csv
